# Data Explory 

In [1]:
#import library 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
from sklearn.metrics import classification_report,ConfusionMatrixDisplay,confusion_matrix

In [2]:
# Load the training data and testing data 
train_data = pd.read_csv('deeptweets/train.csv')

test_data = pd.read_csv('deeptweets/test.csv')

In [3]:
train_data.groupby('Label').describe()

TweetId                                                          \
           count          mean           std           min           25%   
Label                                                                      
Politics  3200.0  2.940386e+17  1.414010e+16  2.477269e+17  2.885772e+17   
Sports    3325.0  2.835879e+17  7.027798e+16  2.390931e+10  2.998319e+17   

                                                    
                   50%           75%           max  
Label                                               
Politics  3.005219e+17  3.042852e+17  3.068234e+17  
Sports    3.043017e+17  3.059188e+17  3.068341e+17

In [4]:
tweets = train_data.copy()
tweets


,TweetId,Label,TweetText
0,304271250237304833,Politics,'#SecKerry: The value of the @StateDept and @U...
1,304834304222064640,Politics,'@rraina1481 I fear so'
2,303568995880144898,Sports,'Watch video highlights of the #wwc13 final be...
3,304366580664528896,Sports,'RT @chelscanlan: At Nitro Circus at #AlbertPa...
4,296770931098009601,Sports,'@cricketfox Always a good thing. Thanks for t...
...,...,...,...
6520,296675082267410433,Politics,'Photo: PM has laid a wreath at Martyrs Monume...
6521,306677536195231746,Sports,'The secret of the Chennai pitch - crumbling o...
6522,306451295307431937,Sports,@alinabhutto he isn't on Twitter either
6523,306088574221176832,Sports,'Which England player would you take out to di...


In [5]:

# missing values
tweets.isna().sum()

TweetId      0
Label        0
TweetText    0
dtype: int64

# data preprocessing

In [6]:
import re
import string
#import emoji

def clean_tweet(tweet):
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    
    # Remove HTML tags
    tweet = re.sub(r'<.*?>', '', tweet)
    
    # Remove special characters and punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)
    
    # Remove extra whitespaces
    tweet = ' '.join(tweet.split())
    
    # Remove mentions and hashtags
    tweet = re.sub(r'@\w+|#\w+', '', tweet)
    
    return tweet



In [7]:
tweets['TweetClean'] = tweets['TweetText'].apply(clean_tweet)

In [8]:
tweets

,TweetId,Label,TweetText,TweetClean
0,304271250237304833,Politics,'#SecKerry: The value of the @StateDept and @U...,SecKerry The value of the StateDept and USAID ...
1,304834304222064640,Politics,'@rraina1481 I fear so',rraina1481 I fear so
2,303568995880144898,Sports,'Watch video highlights of the #wwc13 final be...,Watch video highlights of the wwc13 final betw...
3,304366580664528896,Sports,'RT @chelscanlan: At Nitro Circus at #AlbertPa...,RT chelscanlan At Nitro Circus at AlbertPark t...
4,296770931098009601,Sports,'@cricketfox Always a good thing. Thanks for t...,cricketfox Always a good thing Thanks for the ...
...,...,...,...,...
6520,296675082267410433,Politics,'Photo: PM has laid a wreath at Martyrs Monume...,Photo PM has laid a wreath at Martyrs Monument...
6521,306677536195231746,Sports,'The secret of the Chennai pitch - crumbling o...,The secret of the Chennai pitch crumbling on t...
6522,306451295307431937,Sports,@alinabhutto he isn't on Twitter either,alinabhutto he isnt on Twitter either
6523,306088574221176832,Sports,'Which England player would you take out to di...,Which England player would you take out to din...


# Feature extraction

In [10]:
X_train = tweets['TweetText']
y_train = tweets['Label']
# Feature extraction using TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Model Training

In [11]:
# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


LogisticRegression()

# Model Prediction

In [12]:
# Make predictions on the test data
X_test = test_data['TweetText'].apply(clean_tweet)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
y_pred = model.predict(X_test_tfidf)

y_train_pred = model.predict(X_train_tfidf)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

Training Accuracy: 0.9437547892720306


# submission

In [13]:
# create submission file
submission_df = pd.DataFrame({'TweetId': test_data['TweetId'], 'Label': y_pred})
submission_df.to_csv('submission_f.csv', index=False)

# 5. Additional Approaches
If more time is available, you can consider the following approaches:
* Experiment with different feature extraction techniques (e.g., word embeddings, BERT embeddings).
* Try different machine learning models (e.g., Random Forest, SVM, deep learning models).
* Perform data augmentation for text data.
* Address class imbalance issues using techniques like SMOTE.
* Explore ensemble methods to combine multiple models for improved performance.


In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow

# Function to clean tweets
def clean_tweet(tweet):
    # Your cleaning logic here
    return tweet

# Load the training data and testing data
train_data = pd.read_csv('deeptweets/train.csv')
test_data = pd.read_csv('deeptweets/training.csv')

X_train = train_data['TweetText']
y_train = train_data['Label']

# Feature extraction using TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Models to compare
models = {
    'LogisticRegression': LogisticRegression(),
    'RandomForest': RandomForestClassifier(),
    'MultinomialNB': MultinomialNB()
}

for model_name, model in models.items():
    # Start a new MLflow run for each model
    with mlflow.start_run():
        # Train the model
        model.fit(X_train_tfidf, y_train)

        # Make predictions on the test data
        X_test = test_data['TweetText'].apply(clean_tweet)
        X_test_tfidf = tfidf_vectorizer.transform(X_test)
        y_pred = model.predict(X_test_tfidf)

        # Calculate metrics
        train_accuracy = accuracy_score(y_train, model.predict(X_train_tfidf))
        report = classification_report(test_data['Label'], y_pred)
        conf_matrix = confusion_matrix(test_data['Label'], y_pred)

        # Log parameters, metrics, and artifacts to MLflow
        mlflow.log_params({'max_features': 1000, 'model': model_name})
        mlflow.log_metrics({f'{model_name} Training Accuracy': train_accuracy})
        mlflow.log_artifact('deeptweets/train.csv')
        mlflow.log_artifact('deeptweets/test.csv')
        mlflow.log_text(report, f"{model_name}_classification_report.txt")
        mlflow.log_text(str(conf_matrix), f"{model_name}_confusion_matrix.txt")


In [22]:
mlflow.end_run()

In [ ]:
!mlflow ui -p 1234

[2024-01-05 22:09:11 +0100] [67206] [INFO] Starting gunicorn 21.2.0
[2024-01-05 22:09:11 +0100] [67206] [INFO] Listening at: http://127.0.0.1:1234 (67206)
[2024-01-05 22:09:11 +0100] [67206] [INFO] Using worker: sync
[2024-01-05 22:09:11 +0100] [67207] [INFO] Booting worker with pid: 67207
[2024-01-05 22:09:11 +0100] [67208] [INFO] Booting worker with pid: 67208
[2024-01-05 22:09:12 +0100] [67209] [INFO] Booting worker with pid: 67209
[2024-01-05 22:09:12 +0100] [67210] [INFO] Booting worker with pid: 67210
